In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from typing_extensions import TypedDict

class AgentState(TypedDict):
    query: str
    answer: str
    tax_base_equation: str
    tax_deduction: str
    market_ratio: str
    tax_base: str

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')

vector_store = Chroma(
    embedding_function = embeddings,
    collection_name = 'real_estate_tax2',
    persist_directory = './real_estate_tax_collection2'
)

retriever = vector_store.as_retriever(search_kwargs={'k':3})

In [4]:
query = '5억 짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 경우, 세금은 얼마나 내나요?'

In [5]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = hub.pull('rlm/rag-prompt')

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')


In [ ]:
tax_base_retrieval_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | rag_prompt 
    | llm 
    | StrOutputParser()
)

tax_base_equation_prompt = ChatPromptTemplate.from_messages([
    ('system', '사용자의 질문에서 과세표준을 계산하는 방법을 수식으로 나타내주세요. 부연설명 없이 수식만 리턴해주세요.'),
    ('human', '{tax_base_equation_information}')
])

tax_base_equation_chain = (
    {"tax_base_equation_information": RunnablePassthrough()}
    | tax_base_equation_prompt
    | llm 
    | StrOutputParser()
)

tax_base_chain = {'tax_base_equation_information': tax_base_retrieval_chain} | tax_base_equation_chain

def get_tax_base_equation(state: AgentState) -> str:
    tax_base_equation_quesiton = '주택에 대한 종합 부동산세 계산시 과세표준을 계산하는 방법을 수식으로 표현해서 알려주세요'
    tax_base_equation = tax_base_chain.invoke(tax_base_equation_quesiton)
    return {'tax_base_equation':tax_base_equation}

In [7]:
get_tax_base_equation({})

{'tax_base_equation': '과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율'}

In [8]:
tax_deduction_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

def get_tax_deduction(state: AgentState):
    tax_deduction_question = '주택에 대한 종합부동산세 계산시 공제금액을 알려주세요'
    tax_deduction = tax_deduction_chain.invoke(tax_deduction_question)
    return {'tax_deduction': tax_deduction}


In [9]:
get_tax_deduction({})

{'tax_deduction': '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원, 법인 또는 법인으로 보는 단체의 경우 6억 원, 그 외의 경우 9억 원입니다.'}

In [ ]:
from langchain_community.tools import TavilySearchResults
from datetime import date 

tavily_search_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True
)

tax_market_ratio_prompt = ChatPromptTemplate.from_messages([
    ('system', f'아래 정보를 기반으로 공정시장 가액비율을 계산해주세요 \n\nContext:\n {{context}}'),
    ('human', '{query}')
])


def get_market_ratio(state: AgentState):
    query = f'오늘 날짜:({date.today()})에 해당하는 주택 공시가격 공정시장가액비율은 몇 퍼센트 인가요?'
    context = tavily_search_tool.invoke(query)
    print(f'context: {context}')
    
    tax_market_ratio_chain = (
        tax_market_ratio_prompt
        | llm
        | StrOutputParser()
    )
    
    market_ratio = tax_market_ratio_chain.invoke({'context': context, 'query': query})
    
    return {'market_ratio': market_ratio}

In [39]:
get_market_ratio({})

context: [{'url': 'https://realdecode.com/entry/%EB%B6%80%EC%9E%90%EA%B0%90%EC%84%B8-%EC%A0%95%EC%83%81%ED%99%94-vs-%EC%84%B8%EC%9E%85%EC%9E%90-%EB%B6%80%EB%8B%B4-%EC%A0%84%EA%B0%80%E2%80%A6-%EC%A2%85%EB%B6%80%EC%84%B8-%EA%B0%9C%ED%8E%B8-%ED%95%B5%EC%8B%AC-%EC%9F%81%EC%A0%90-%EC%B4%9D%EC%A0%95%EB%A6%AC?category=1257686', 'content': '공정시장가액비율은 주택이나 토지의 공시가격에 곱해 종부세 과세표준을 산정하는 계수입니다. 현행 법령상 60~100% 범위 내에서 정부 시행령으로 자유롭게 조정이 가능합니다.\n\n 문재인 정부: 최대 95%까지 인상\n 윤석열 정부: 60%로 대폭 하향 조정\n 이재명 정부: 2025년 80%로 상향 검토 중\n\n공정시장가액비율이 높아지면 과세표준이 커지고, 결과적으로 종부세 부담이 증가합니다. 이에 따라 부동산 시장은 물론 임대시장, 세입자 가계에도 직간접적 영향을 미칠 수 있습니다.\n\n## 2. 고가 주택 보유세, 얼마나 오를까\n\n전문가들이 시뮬레이션한 바에 따르면, 공시가격과 공정시장가액비율이 동시에 상승할 경우 보유세(재산세+종부세)는 최대 50%까지 증가할 수 있습니다.\n\n서울 강남 3구뿐 아니라 마포, 용산, 성동 등 인기 지역 전반으로 이 같은 보유세 상승이 확산될 가능성이 있습니다.\n\n## \n\n## 3. 보유세 인상이 공급을 늘릴까? [...] 공정시장가액비율 인상에 더해, 공시가격 현실화 정책까지 재추진될 가능성이 제기되고 있습니다.\n\n 문재인 정부: 2030년까지 공시가격 시세 90% 목표\n 윤석열 정부: 현실화 로드맵 백지화\n 2025년 이후: 단계적 재도입 검토 예상\n\n공시가격 인상과 공정시장가액비율 인상이 병행되면, 국

{'market_ratio': '2025년 기준으로 주택 공시가격에 적용되는 공정시장가액비율은 60%입니다.'}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

tax_base_calculation_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', """ 
            주어진 내용을 기반으로 과세표준을 계산해주세요.
            과세표준 계산 공식 {tax_base_equation}
            공제금액: {tax_deduction}
            공정시장가액비율: {market_ratio}                                                
        """),
        ('human', '사용자 주택 공시가격 정보: {query}')
    ]
   )


def calculate_tax_base(state: AgentState):
    tax_base_equation = state['tax_base_equation']
    tax_deduction = state['tax_deduction']
    market_ratio = state['market_ratio']
    query = state['query']
    
    tax_base_calculation_chain = (
        tax_base_calculation_prompt
        | llm
        | StrOutputParser()
    )
    
    tax_base = tax_base_calculation_chain.invoke({
        'tax_base_equation': tax_base_equation,
        'tax_deduction': tax_deduction,
        'market_ratio': market_ratio,
        'query': query
    })
    
    print(f'tax_base: {tax_base}')
    
    return {'tax_base': tax_base}

In [12]:
initial_state = {
    'query': query,
    'tax_base_equation': '과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율',
    'tax_deduction': '주택에 대한 종합부동산세 계산 시 공제금액은 1세대 1주택자의 경우 12억 원, 법인 또는 법인으로 보는 단체는 6억 원, 그 외의 경우는 9억 원입니다.',
    'market_ratio': '주어진 정보를 바탕으로 2025년의 주택 공시가격 공정시장가액비율은 60%입니다.'
}

calculate_tax_base(initial_state)

tax_base: 주어진 정보를 바탕으로 과세표준을 계산해보겠습니다.

1. 주택 공시가격 합계:
   - 5억 원 + 10억 원 + 20억 원 = 35억 원

2. 공제금액:
   - 사용자에 대한 추가 정보가 없으므로, 일반적인 경우인 9억 원을 공제금액으로 사용하겠습니다.

3. 공정시장가액비율:
   - 2025년의 공정시장가액비율은 60%입니다.

과세표준 계산:
- 과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율
- 과세표준 = (35억 원 - 9억 원) × 60%
- 과세표준 = 26억 원 × 60%
- 과세표준 = 15.6억 원

따라서, 과세표준은 15.6억 원입니다. 이 과세표준을 바탕으로 종합부동산세 등급에 따른 세율을 적용하여 최종 세금을 계산해야 합니다. 세율 정보가 필요하다면 추가로 제공해 드릴 수 있습니다.


{'tax_base': '주어진 정보를 바탕으로 과세표준을 계산해보겠습니다.\n\n1. 주택 공시가격 합계:\n   - 5억 원 + 10억 원 + 20억 원 = 35억 원\n\n2. 공제금액:\n   - 사용자에 대한 추가 정보가 없으므로, 일반적인 경우인 9억 원을 공제금액으로 사용하겠습니다.\n\n3. 공정시장가액비율:\n   - 2025년의 공정시장가액비율은 60%입니다.\n\n과세표준 계산:\n- 과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율\n- 과세표준 = (35억 원 - 9억 원) × 60%\n- 과세표준 = 26억 원 × 60%\n- 과세표준 = 15.6억 원\n\n따라서, 과세표준은 15.6억 원입니다. 이 과세표준을 바탕으로 종합부동산세 등급에 따른 세율을 적용하여 최종 세금을 계산해야 합니다. 세율 정보가 필요하다면 추가로 제공해 드릴 수 있습니다.'}

In [ ]:
tax_rate_calculation_prompt = ChatPromptTemplate.from_messages([
    ('system', '''당신은 종합부동산세 계산 전문가 입니다. 아래 문서를 참고해서 사용자의 질문에 대한 종합부동산세를 계산해주세요 
     종합 부동산세 세율: {context}'''),
    
    ('human', '''과세표준과 사용자가 소지한 주택의 수가 아래와 같을때 종합부동산세를 계산해주세요 
     과세표준: {tax_base}
     주택 수: {query}
     ''')
])

def calculate_tax_rate(state: AgentState):
    query = state['query']
    tax_base = state['tax_base']
    context = retriever.invoke(query)
    
    tax_rate_chain = (
        tax_rate_calculation_prompt
        | llm
        | StrOutputParser()
    )
    
    tax_rate = tax_rate_chain.invoke({
        'context': context, 
        'tax_base': tax_base, 
        'query': query
    })

    return {'answer': tax_rate}

In [15]:
tax_base_state = {'query': query, 'tax_base': '주어진 정보를 바탕으로 사용자의 과세표준을 계산해보겠습니다.\n\n1. 주택 공시가격의 합계:\n   - 5억 원 + 10억 원 + 20억 원 = 35억 원\n\n2. 공제금액:\n   - 사용자가 1세대 1주택자가 아니므로, 공제금액은 9억 원입니다.\n\n3. 과세표준 계산:\n   - 과세표준 = (주택 공시가격 합계 - 공제금액) × 공정시장가액비율\n   - 과세표준 = (35억 원 - 9억 원) × 60%\n   - 과세표준 = 26억 원 × 60%\n   - 과세표준 = 15.6억 원\n\n따라서 사용자의 과세표준은 15.6억 원입니다. 과세표준에 따라 구체적인 세율을 적용하여 세금을 계산할 수 있습니다. 세율은 주택 보유 유형과 과세표준 금액에 따라 다르게 적용됩니다.'}

calculate_tax_rate(tax_base_state)

{'answer': '주어진 정보를 바탕으로 사용자의 종합부동산세를 계산해보겠습니다. 사용자는 총 3채의 주택을 소유하고 있으므로, 3주택 이상 소유한 경우의 세율을 적용해야 합니다. 사용자의 과세표준은 15.6억 원입니다.\n\n3주택 이상 소유한 경우의 과세표준과 세율은 다음과 같습니다:\n- 12억 원 초과 25억 원 이하: 960만 원 + (12억 원을 초과하는 금액의 1천분의 20)\n\n과세표준이 15.6억 원이므로, 12억 원을 초과하는 금액은:\n- 15.6억 원 - 12억 원 = 3.6억 원\n\n따라서, 종합부동산세는 다음과 같이 계산됩니다:\n- 세금 = 960만 원 + (3.6억 원 × 0.002)\n- 세금 = 960만 원 + 720만 원\n- 세금 = 1,680만 원\n\n따라서 사용자는 1,680만 원의 종합부동산세를 납부해야 합니다.'}

In [32]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(AgentState)

In [33]:
graph_builder.add_node('get_tax_base_equation', get_tax_base_equation)
graph_builder.add_node('get_tax_deduction', get_tax_deduction)
graph_builder.add_node('get_market_ratio', get_market_ratio)
graph_builder.add_node('calculate_tax_base', calculate_tax_base)
graph_builder.add_node('calculate_tax_rate', calculate_tax_rate)


In [34]:
from langgraph.graph import START, END

graph_builder.add_edge(START, 'get_tax_base_equation')
graph_builder.add_edge(START, 'get_tax_deduction')
graph_builder.add_edge(START, 'get_market_ratio')
graph_builder.add_edge('get_tax_base_equation', 'calculate_tax_base')
graph_builder.add_edge('get_tax_deduction', 'calculate_tax_base')
graph_builder.add_edge('get_market_ratio', 'calculate_tax_base')
graph_builder.add_edge('calculate_tax_base', 'calculate_tax_rate')
graph_builder.add_edge('calculate_tax_rate', END)


In [35]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [37]:
initial_state = {'query': query}

graph.invoke(initial_state)

context: [{'url': 'https://www.law.go.kr/LSW/lumLsLinkPop.do?lspttninfSeq=120262', 'content': '1. 지방세법 시행령  \n[시행 2025. 11. 25.] [대통령령 제35858호, 2025. 11. 25., 타법개정]\n\n 제109조 공정시장가액비율\n\n## 지방세법 시행령\n\n[시행 2025. 11. 25.] [대통령령 제35858호, 2025. 11. 25., 타법개정]\n\n  \n\n① 법 제110조제1항 각 호 외의 부분에서 “대통령령으로 정하는 공정시장가액비율”이란 다음 각 호의 구분에 따른 비율을 말한다. <개정 2022. 6. 30., 2023. 3. 14., 2023. 6. 30., 2024. 5. 28., 2025. 5. 27.>\n\n1. 토지 및 건축물: 시가표준액의 100분의 70\n\n2. 주택: 시가표준액의 100분의 60. 다만, 2025년도에 납세의무가 성립하는 재산세의 과세표준을 산정하는 경우 제110조의2에 따라 1세대 1주택으로 인정되는 주택(시가표준액이 9억원을 초과하는 주택을 포함한다)에 대해서는 다음 각 목의 구분에 따른다. [...] 가. 시가표준액이 3억원 이하인 주택: 시가표준액의 100분의 43\n\n나. 시가표준액이 3억원을 초과하고 6억원 이하인 주택: 시가표준액의 100분의 44\n\n다. 시가표준액이 6억원을 초과하는 주택: 시가표준액의 100분의 45\n\n② 행정안전부장관은 제1항에 따른 공정시장가액비율의 점검ㆍ평가를 위하여 필요한 경우 관계 전문기관에 조사ㆍ연구를 의뢰할 수 있다. <신설 2023. 3. 14.>\n\n 파일형식\n\n## 지방세법 시행령\n\n[시행 2025. 11. 25.] [대통령령 제35858호, 2025. 11. 25., 타법개정]\n\n  \n\n① 법 제110조제1항 각 호 외의 부분에서 “대통령령으로 정하는 공정시장가액비율”이란 다음 각 호의 구분에 따른 비율을 말한다. <개정 2022. 

{'query': '5억 짜리 집 1채, 10억짜리 집 1채, 20억짜리 집 1채를 가지고 있을 경우, 세금은 얼마나 내나요?',
 'answer': '사용자가 소유한 주택 수에 따라 종합부동산세 세율을 적용하여 계산해 보겠습니다. 주어진 정보에 따르면 사용자는 총 3채의 주택을 소유하고 있으며, 이 경우 3주택 이상 소유자로 분류됩니다.\n\n과세표준이 15.7억 원일 때, 3주택 이상의 경우에 해당하는 세율을 적용하면 다음과 같습니다:\n\n1. 과세표준 12억 원 초과 25억 원 이하에 해당하므로,\n   - 기본 세액: 960만 원\n   - 추가 세액: (15.7억 원 - 12억 원) × 1천분의 20 = 3.7억 원 × 0.002 = 740만 원\n\n따라서, 종합부동산세는 다음과 같이 계산됩니다:\n\n- 기본 세액: 960만 원\n- 추가 세액: 740만 원\n- 총 세액: 960만 원 + 740만 원 = 1,700만 원\n\n따라서, 사용자는 총 1,700만 원의 종합부동산세를 납부해야 합니다.',
 'tax_base_equation': '과세표준 = (주택의 공시가격 합계 - 공제금액) × 공정시장가액비율',
 'tax_deduction': '주택에 대한 종합부동산세 계산 시, 1세대 1주택자는 12억 원, 법인이나 법인으로 보는 단체는 6억 원, 그 외의 경우는 9억 원이 공제됩니다.',
 'market_ratio': '2025년의 주택 공정시장가액비율은 다음과 같습니다:\n\n- 일반적인 주택: 시가표준액의 60%\n- 1세대 1주택으로 인정되는 주택(시가표준액이 9억원을 초과하는 주택 포함)의 경우:\n  - 시가표준액이 3억원 이하인 주택: 시가표준액의 43%\n  - 시가표준액이 3억원을 초과하고 6억원 이하인 주택: 시가표준액의 44%\n  - 시가표준액이 6억원을 초과하는 주택: 시가표준액의 45%',
 'tax_base': '주어진 정보를 바탕으로 주택의 과세표준을 계산해보겠습니다. \n\n사용자의 주택 공시가격 합계는:\n- 